## Load

In [1]:
from scipy.io import loadmat
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go


mat = loadmat('signals.mat')
data = {
    key:np.array(mat[key].T, dtype=float)
    for key in mat.keys() if not key.startswith('__')
}

assert np.allclose(data["ICOB"]*(data["COB"]+0.1), data["IOB"])
data.update({"DCGM": data["DCOB"]*(data["COB"]+0.1)})
# def smooth(signal, w_size=60//5):
#     return np.convolve(signal, np.ones(w_size)/w_size)[:len(signal)]
# df.update(
#     {"DCGM": np.apply_along_axis(smooth, 1, np.diff(df["CGM"], axis=1, prepend=0))}
# )

def get_fault_progress(signal):
    starts = np.where(np.diff(signal)<0)[0]
    ends = np.where(np.diff(signal)>0)[0]
    fault_progress = np.zeros_like(signal)
    for start, end in zip(starts, ends):
        fault_progress[start:end+(end-start)] = np.linspace(0, 2*(end-start)*5/60, 2*(end-start))
    return fault_progress

def get_fault_state(signal):
    starts = np.where(np.diff(signal)<0)[0]
    ends = np.where(np.diff(signal)>0)[0]
    fault_state = np.zeros(signal.shape, dtype=int)
    for start, end in zip(starts, ends):
        fault_state[start:end] = 1
        fault_state[end:end+(end-start)] = -1
    fault_state = np.array(["Recovery", "Healthy", "Faulty"])[fault_state+1]
    return fault_state

fault_progress = np.apply_along_axis(get_fault_progress, 1, data["FaultsPump"])
fault_state = np.apply_along_axis(get_fault_state, 1, data["FaultsPump"])
data.update({"FaultState": fault_state, "FaultProgress": fault_progress})

df_patients = [
    pd.DataFrame(
        {key:signal[patient_id] for key, signal in data.items()}
    )
    for patient_id in range(data["CGM"].shape[0])
]
for df in df_patients:
    df["Time"] = pd.date_range("2000-01-01", periods=df.shape[0], freq="5min")

ModuleNotFoundError: No module named 'pandas'

# Signals

Dumb Questions:
1. IOB (and ICOB) are 0 untill 3h
2. ICOB = IOB/(COB+eps) -> i don't remember why eps=0.1
3. DCOB = diff(CGM)/(COB+eps) + smoothing -> how is it smoothed


prop
CGM/COB

In [ ]:
patient_id = 27

In [ ]:
px.line(
    df_patients[patient_id], x="Time", 
    y=['CGM', 'CHO', 'COB', 'DCOB', 'ICOB', 'IOB', 'insulin', 'DCGM', 'FaultsPump', 'FaultProgress'],
).show()

# plot faults divided by FaultState
px.scatter(
    df_patients[patient_id], x="Time", y='FaultProgress',
    color="FaultState"
).show()

# 3d Scatters

## Normalized

In [ ]:
fig = px.scatter_3d(
    df_patients[patient_id], x="CGM", y="ICOB", z="DCOB",
    color="FaultProgress", color_continuous_scale="Hot", 
    range_color=[
        df_patients[patient_id]["FaultProgress"].min(), 
        df_patients[patient_id]["FaultProgress"].max()
    ],
    symbol="FaultState", opacity=0.5, height=600, width=600,
)
fig.update_traces(
    marker=dict(
        line_width=0,
        size=np.ones_like(df_patients[patient_id]["FaultProgress"])*5
    )
)
fig.update_layout(
    legend=dict(yanchor="top",y=0.99,xanchor="left",x=0.01)
)
fig.show()

## Unormalized

Fault = High IOB, Low COB, positive DCGM (but is not well separated)

In [ ]:
fig = px.scatter_3d(
    df_patients[patient_id], x="DCGM", y="IOB", z="COB",
    color="FaultProgress", color_continuous_scale="Hot", 
    range_color=[
        df_patients[patient_id]["FaultProgress"].min(), 
        df_patients[patient_id]["FaultProgress"].max()
    ],
    symbol="FaultState", opacity=0.5, height=600, width=600,
)
fig.update_traces(
    marker=dict(
        line_width=0,
        size=np.ones_like(df_patients[patient_id]["FaultProgress"])*5
    )
)
fig.update_layout(
    legend=dict(yanchor="top",y=0.99,xanchor="left",x=0.01)
)
fig.show()

my understanding of why normalization works

IOB -> tries to lower CGM

COB -> tries to increase CGM

$\Delta \text{CGM} \approx c_\text{COB} \text{COB} - c_\text{IOB} \text{IOB}$

here an hyperplane separates abnormal from normal behaviour (Delta higher/lower than expected)

if we normalize (divide by COB) we get 

$\text{DCOB} \approx c_\text{COB} - c_\text{IOB} \text{ICOB}$

here the same 3D relation is condensed into 2D, the plane is now a line